# BANK LOAN OPTIMIZATION PROJECT

## PROBLEM IDENTIFICATION
    -Data loading

    Basic analysis-

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_loan_data(num_applications=1000):
    np.random.seed(42)
    
    data = {
        'application_id': range(1, num_applications + 1),
        'application_date': pd.date_range('2023-01-01', periods=num_applications),
        'customer_age': np.random.randint(25, 65, num_applications),
        'annual_income': np.random.normal(60000, 25000, num_applications).clip(20000, 200000),
        'loan_amount': np.random.uniform(5000, 100000, num_applications),
        'loan_purpose': np.random.choice(['debt_consolidation', 'home_improvement', 'business', 
                                        'personal', 'car_purchase'], num_applications),
        'employment_length': np.random.choice(['<1 year', '1-3 years', '3-5 years', '5-10 years', '10+ years'], num_applications),
        'credit_score': np.random.randint(300, 850, num_applications),
        'dti_ratio': np.random.uniform(5, 40, num_applications),  # Debt-to-income ratio
        'existing_loans': np.random.randint(0, 5, num_applications)
    }
    
    df = pd.DataFrame(data)
    
    # Simulate process timing data
    df['processing_days'] = np.random.randint(3, 20, num_applications)
    df['status'] = np.random.choice(['approved', 'rejected', 'pending'], num_applications, p=[0.6, 0.3, 0.1])
    
    return df

# Generate and save data
loan_data = generate_loan_data(1000)
loan_data.to_csv('loan_applications.csv', index=False)

In [2]:
df = pd.read_csv("loan_applications.csv")

In [3]:
df.head()

,application_id,application_date,customer_age,annual_income,loan_amount,loan_purpose,employment_length,credit_score,dti_ratio,existing_loans,processing_days,status
0,1,2023-01-01,63,44314.527891,97915.872707,business,10+ years,744,5.417129,1,8,approved
1,2,2023-01-02,53,47615.268927,80429.023055,home_improvement,10+ years,321,9.889233,1,13,approved
2,3,2023-01-03,39,70470.985347,52239.313380,car_purchase,1-3 years,590,30.050757,1,19,approved
3,4,2023-01-04,32,50267.795207,37278.859798,home_improvement,10+ years,784,39.047780,2,11,approved
4,5,2023-01-05,45,29126.390162,87852.891502,business,5-10 years,605,39.407264,3,19,rejected


# Data Analysis & Insights Generation

In [5]:

# Loading the  data and doing  initial exploration
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  data loading 
loan_df = pd.read_csv('loan_applications.csv')

# Basic exploration
print("=== DATA OVERVIEW ===")
print(f"Dataset shape: {loan_df.shape}")
print("\nBasic info:")
print(loan_df.info())
print("\nFirst 5 rows:")
print(loan_df.head())

print("\n=== KEY STATISTICS ===")
print(loan_df.describe())

print("\n=== STATUS DISTRIBUTION ===")
print(loan_df['status'].value_counts())

=== DATA OVERVIEW ===
Dataset shape: (1000, 12)

Basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   application_id     1000 non-null   int64  
 1   application_date   1000 non-null   object 
 2   customer_age       1000 non-null   int64  
 3   annual_income      1000 non-null   float64
 4   loan_amount        1000 non-null   float64
 5   loan_purpose       1000 non-null   object 
 6   employment_length  1000 non-null   object 
 7   credit_score       1000 non-null   int64  
 8   dti_ratio          1000 non-null   float64
 9   existing_loans     1000 non-null   int64  
 10  processing_days    1000 non-null   int64  
 11  status             1000 non-null   object 
dtypes: float64(3), int64(5), object(4)
memory usage: 93.9+ KB
None

First 5 rows:
   application_id application_date  customer_age  annual_income   loan_amount 

In [6]:
# Analysis 1: Processing time by loan purpose
processing_by_purpose = loan_df.groupby('loan_purpose').agg({
    'processing_days': ['mean', 'median', 'count'],
    'status': lambda x: (x == 'approved').mean() * 100  # approval rate
}).round(2)

print("Processing Time & Approval Rate by Loan Purpose:")
print(processing_by_purpose)

# Analysis 2: Credit score impact on approval
credit_analysis = loan_df.groupby(pd.cut(loan_df['credit_score'], bins=[300, 580, 670, 740, 800, 850])).agg({
    'status': lambda x: (x == 'approved').mean() * 100,
    'processing_days': 'mean'
}).round(2)

print("\nApproval Rate by Credit Score Tier:")
print(credit_analysis)

Processing Time & Approval Rate by Loan Purpose:
                   processing_days                status
                              mean median count <lambda>
loan_purpose                                            
business                     10.95   11.0   208    62.98
car_purchase                 10.22   10.0   210    59.05
debt_consolidation           10.61   11.0   198    65.15
home_improvement             11.05   11.0   197    64.47
personal                     11.39   12.0   187    60.96

Approval Rate by Credit Score Tier:
              status  processing_days
credit_score                         
(300, 580]     61.51            10.75
(580, 670]     64.58            11.15
(670, 740]     58.06            10.69
(740, 800]     71.57            11.38
(800, 850]     61.22            10.46


C:\Users\asus\AppData\Local\Temp\ipykernel_7416\3183211181.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  credit_analysis = loan_df.groupby(pd.cut(loan_df['credit_score'], bins=[300, 580, 670, 740, 800, 850])).agg({


# ROOT CAUSE ANALYSIS  

In [7]:
# Simple benefit calculation
current_time = loan_df['processing_days'].mean()
new_time = current_time - 3  # Save 3 days

print(f"Current: {current_time:.1f} days")
print(f"Target: {new_time:.1f} days") 
print(f"Improvement: {((current_time - new_time)/current_time*100):.1f}% faster")

Current: 10.8 days
Target: 7.8 days
Improvement: 27.7% faster


In [8]:
# Load your data
loan_df = pd.read_csv('loan_applications.csv')

#  answer  of these 3 questions:
print("1. How many applications? ", len(loan_df))
print("2. Average processing days? ", loan_df['processing_days'].mean())
print("3. How many take more than 15 days? ", len(loan_df[loan_df['processing_days'] > 15]))



1. How many applications?  1000
2. Average processing days?  10.829
3. How many take more than 15 days?  219


# Key Findings

In [11]:
print(" KEY INSIGHTS for BANK:")
print(f"• We analyzed 1,000 loan applications")
print(f"• Average processing time: 10.8 days") 
print(f"• 219 applications (22%) take >15 days - THIS IS THE PROBLEM!")
print(f"• Almost 1 in 4 applications are too slow")

 KEY INSIGHTS for BANK:
• We analyzed 1,000 loan applications
• Average processing time: 10.8 days
• 219 applications (22%) take >15 days - THIS IS THE PROBLEM!
• Almost 1 in 4 applications are too slow


In [10]:
# Let's find what makes applications slow
slow_apps = loan_df[loan_df['processing_days'] > 15]
fast_apps = loan_df[loan_df['processing_days'] <= 10]

print("=== WHY ARE APPLICATIONS SLOW? ===")
print("\n1. SLOW applications characteristics:")
print(f"   • Average loan amount: ${slow_apps['loan_amount'].mean():.0f}")
print(f"   • Average credit score: {slow_apps['credit_score'].mean():.0f}")
print(f"   • Most common purpose: {slow_apps['loan_purpose'].mode().values[0]}")

print("\n2. FAST applications characteristics:")
print(f"   • Average loan amount: ${fast_apps['loan_amount'].mean():.0f}") 
print(f"   • Average credit score: {fast_apps['credit_score'].mean():.0f}")
print(f"   • Most common purpose: {fast_apps['loan_purpose'].mode().values[0]}")

print("\n3. COMPARISON:")
print(f"   • Slow apps have ${slow_apps['loan_amount'].mean() - fast_apps['loan_amount'].mean():.0f} higher loans")
print(f"   • Fast apps have {fast_apps['credit_score'].mean() - slow_apps['credit_score'].mean():.0f} better credit scores")

=== WHY ARE APPLICATIONS SLOW? ===

1. SLOW applications characteristics:
   • Average loan amount: $52232
   • Average credit score: 582
   • Most common purpose: home_improvement

2. FAST applications characteristics:
   • Average loan amount: $53670
   • Average credit score: 571
   • Most common purpose: car_purchase

3. COMPARISON:
   • Slow apps have $-1438 higher loans
   • Fast apps have -11 better credit scores


# SOLUTION DESIGN


In [14]:
#  DESIGN THE SOLUTION BASED ON OUR FINDINGS
print("=== OUR SOLUTION: PROCESS OPTIMIZATION ===")

# Solution 1: Implement Tiered Processing
print("\n SOLUTION 1: TIERED PROCESSING SYSTEM")
print("   • FAST TRACK: Credit score > 700 → Auto-approve in 3 days")
print("   • STANDARD: Credit score 600-700 → Process in 7 days") 
print("   • COMPREHENSIVE: Credit score < 600 → Full review in 10 days")

# Solution 2: Parallel Processing
print("\n SOLUTION 2: PARALLEL PROCESSING")
print("   CURRENT: Application → Docs → Credit → Risk → Underwrite → Approve")
print("   IMPROVED: Application → Docs+Credit (together) → Risk → Underwrite+Approve (together)")

# Calculate the improvement
current_avg = loan_df['processing_days'].mean()
proposed_avg = 7.0  # Our target

print(f"\n EXPECTED IMPROVEMENT:")
print(f"   Current average: {current_avg} days")
print(f"   Target average: {proposed_avg} days")
print(f"   Reduction: {current_avg - proposed_avg} days ({(current_avg - proposed_avg)/current_avg*100:.1f}% faster)")

# Show how many applications would benefit
fast_track_apps = loan_df[loan_df['credit_score'] > 700]
print(f"\n IMPACT ANALYSIS:")
print(f"   {len(fast_track_apps)} applications would qualify for FAST TRACK")
print(f"   {len(slow_apps)} slow applications would be eliminated")

=== OUR SOLUTION: PROCESS OPTIMIZATION ===

 SOLUTION 1: TIERED PROCESSING SYSTEM
   • FAST TRACK: Credit score > 700 → Auto-approve in 3 days
   • STANDARD: Credit score 600-700 → Process in 7 days
   • COMPREHENSIVE: Credit score < 600 → Full review in 10 days

 SOLUTION 2: PARALLEL PROCESSING
   CURRENT: Application → Docs → Credit → Risk → Underwrite → Approve
   IMPROVED: Application → Docs+Credit (together) → Risk → Underwrite+Approve (together)

 EXPECTED IMPROVEMENT:
   Current average: 10.829 days
   Target average: 7.0 days
   Reduction: 3.8290000000000006 days (35.4% faster)

 IMPACT ANALYSIS:
   276 applications would qualify for FAST TRACK
   219 slow applications would be eliminated


# Business Case

In [16]:
#  CALCULATE BUSINESS VALUE
print("\n=== BUSINESS CASE ===")

# Simple cost-benefit analysis
applications_per_year = 50000  # Assume Deutsche Bank processes 50,000 loans/year
cost_per_application_day = 50  # Estimated operational cost per day

current_annual_cost = applications_per_year * current_avg * cost_per_application_day
proposed_annual_cost = applications_per_year * proposed_avg * cost_per_application_day
annual_savings = current_annual_cost - proposed_annual_cost

print(f"FINANCIAL IMPACT:")
print(f"   Current annual cost: ${current_annual_cost:,.2f}")
print(f"   Proposed annual cost: ${proposed_annual_cost:,.2f}") 
print(f"   ANNUAL SAVINGS: ${annual_savings:,.2f}")

print(f"\n CUSTOMER IMPACT:")
print(f"   Faster loan decisions → Better customer experience")
print(f"   Competitive advantage vs other banks")


=== BUSINESS CASE ===
FINANCIAL IMPACT:
   Current annual cost: $27,072,500.00
   Proposed annual cost: $17,500,000.00
   ANNUAL SAVINGS: $9,572,500.00

 CUSTOMER IMPACT:
   Faster loan decisions → Better customer experience
   Competitive advantage vs other banks
